In [1]:
import pickle
import pandas as pd
import s3fs
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import json 

In [2]:
asistente=pd.read_csv('s3://uploadtp/BI/archivos_subir/CARTERAS_TXT_2/asistente_virtual.csv000',
                     dtype={'cuenta':'str'})

In [2]:
infantil=pd.read_csv("s3://uploadtp/BI/archivos_subir/CARTERAS_TXT_2/min_infantil_oct.csv000",dtype={'cuenta':str})

In [2]:
mes='sep'
TARGET ={ 
        'TV_adicional':{'use':0, 'Y':'mas_1_tv','variable_extra':False,'input_gen':0,'class':'binary'},
        'Adult_pack':{'use':0, 'Y':'adult_pack', 'variable_extra':'busquedas_porn','input_gen':0,'class':'binary'},
        'Stingray':{'use':0,'Y':'stingray', 'variable_extra':False,'input_gen':0},
        'Qello': {'use':0, 'Y':'qello', 'variable_extra':False,'input_gen':0,'class':'binary'},
        'Blim':{'use':0, 'Y':'blim', 'variable_extra':False,'input_gen':0,'class':'binary'},
        'Picardia':{'use':0, 'Y':'picardia', 'variable_extra':False,'input_gen':0},
        'Fitness':{'use':0, 'Y':'fitness', 'variable_extra':'busquedas_fitness','input_gen':0},
        'Golden':{'use':0, 'Y':'golden', 'variable_extra':False,'input_gen':0,'class':'binary'},
        'Noggin':{'use':0, 'Y':'noggin', 'variale_extra':'min_infantil','input_gen':0,'class':'binary'},
        'Sound_box':{'use':0, 'Y':'sound_box', 'variable_extra':'asistente_virtual', 'input_gen':0,'class':'binary'},
        'Paramount':{'use':0, 'Y':'paramount', 'variable_extra':False,'input_gen':0},
        'Hogar_seguro':{'use':0, 'Y':'hogar_seguro', 'variable_extra':'busquedas_seguridad','input_gen':0},
        'Parrilla':{'use':0, 'Y':'parrilla', 'variable_extra':False,'input_gen':0,'class':'multi'},
        'Starzplay':{'use':1, 'Y':'starzplay', 'variable_extra':False,'input_gen':0,'class':'binary'}}

tar=""
modelo=""
for model in TARGET.keys():
    if TARGET[model]['use']==1:
        modelo=model
        tar=TARGET[model]['Y']
    else:
        pass

In [3]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

print(f"{bcolors.BOLD}Iniciando proceso para {modelo}\n")

Iniciando proceso para Starzplay



In [4]:
moda=['renta','rent','avg_cluster_edad','edad_impute','plan_residencial_negocios',
      'plan_precio_lista','plan_precio_pronto_pago','as_next_fam_plan_play_num','as_previous_rent','plan_megas_de_subida','plan_megas_de_bajada',
      'as_previous_rent','plan_megas_de_subida','plan_megas_de_bajada',]
fill_0=['rent_addons','plan_tv_incluidas','plan_num_diff','usg_siete_canales','usg_abierta','usg_paga','usg_premimum',
        'upsl_nbr_4_overage_margin','up_tot_3p_to_2p','up_tot_2p_to_3p','up_tot_2p_to_2p','up_tot_3p_to_3p','total_promo',
        'new_cons_videogames','new_cons_video','new_cons_ottstreaming','new_cons_musicstreaming','new_cons_socialmedia',
        'new_cons_cloud','new_cons_videoconference','new_cons_netflix','new_cons_primevideo','new_cons_disneyplus',
        'tel_duracion_semana','tel_duracion_fin_semana','tel_morning','tel_afternoon','tel_evening','tel_nigth','tel_overnigth',
        'tel_llamadas_saliente','tel_llamadas_internacional','tel_duracion_internacional_hr','carr_telmex_duracion',
        'carr_izzi_duracion','carr_sky_duracion','carr_dish_duracion','carr_telmex_llamadas','carr_izzi_llamadas','carr_sky_llamadas',
        'carr_dish_llamadas','carr_tot_duracion','carr_tot_llamadas','tot_cons','usg_tv_total','monto_pct_of_rent',
        'ph_plan_tv_incluidas','ph_wifi_addons']
prom=['score','usg_phone_nac_t','usg_n_stb','usg_phone_total_t','usg_tv_anytime','pop_age_18to24_pct','pct_hh_cell_phone',
     'pct_hh_internet','pct_hh_washing_machine','pct_hh_tv_subscription','pct_hh_tv','avg_occupants_per_dwelling',
     'pct_dwelling_1_room','pct_dwelling_2_room','pct_dwelling_3plus_room','new_cons_youtube','stb1','weekday',
      'weekend','morning','noon','evening','night','overnight','noticias','entretenimiento','comedia','telenovela','deportes',
      'otros','ph_add_channels','usg_down_bytes_total','usg_up_bytes_total','usg_down_bytes_total','usg_up_bytes_total',
      'upsl_nbr_3_paquete_margin','dwnsl_nbr_3_paquete_margin']

dropr=['longitud','latitud','approved_plan','family_plan','as_next_fam_plan_cat','usg_phone_int_t',
      'dwnsl_nbr_4_overage_margin','up_sum_margen_crecimiento','up_sum_margen_paquete','up_max_up_megas_change',
     'up_max_down_megas_change','up_max_tv_change','up_max_minutos_nat_change','up_max_minutos_inter_change',
      'up_max_services_inc_change','up_max_precio_pronto_change','up_max_precio_lista_change','new_cons_hbo',
       'fecha_activacion_raw','fecha_activacion','ano_semana','cancel_year','cancel_month','cancelation_week',
       'cancelation_date','nse','edad','plan_internet_incluidas','plan_minutos_nacional'
       ,'plan_minutos_internacional','gpo_edad','tipo_persona','television','anytime','netflix','youtube','amazon','deezer','vod',
       'period','billing_month','as_prev_mth','as_nxt_mth','motive','as_nxt_des_svc','as_next_rent','as_next_total_rent']

In [5]:
def limpieza(data,moda,zeros,promedio,drop):
    data['as_nxt_app_pln'].fillna(data['plan_homologado'],inplace=True)
    data['as_prev_fam_pln'].fillna(data['familia'],inplace=True)
    data['as_nxt_fam_pln'].fillna(data['familia'],inplace=True)
    data['plan_servicios_incluidos'].fillna('NINGUNO',inplace=True)
    for col in moda:
        data[col].fillna(data[col].mode()[0],inplace=True)
    for col in zeros:
        data[col].fillna(0,inplace=True)
    for col in promedio:
        data[col].fillna(data[col].mean(),inplace=True)
    for col in drop:
        data.drop(columns=col,inplace=True)
    for col in data.columns.to_list():
        if data[col].isnull().sum()>=2672000:
            data.drop(columns=col, inplace=True)
    data['plan_megas_de_bajada'].fillna(data['plan_megas'],inplace=True)
    data['plan_megas_de_subida'].fillna(data['plan_megas']/10,inplace=True)
    data.dropna(subset = ["segmento_de_atraso"], inplace=True)    
    data['as_previous_total_rent'].fillna(data['total_rent'],inplace=True)
    data['as_previous_rent'].fillna(data['rent'],inplace=True)
    data['as_prev_app_pln'].fillna(data['plan_homologado'],inplace=True)
    data['as_prev_des_svc'].fillna(data['desc_service'],inplace=True)
    data['ano_act']=data['ano_act'].astype('str')
    data.drop(columns=['addon_promo_category','plan_megas','AAA_megas'],inplace=True)
    
    for col in data.columns.tolist():
        if data[col].isnull().sum()!=0:
            data[col].fillna(data[col].mode()[0],inplace=True)
        else:
            pass
    return data 

#Nos da el label encoding y estandariza las variables
def encoding(data,target):
    num_col = data.columns[(data.dtypes !='object').values].tolist()
    cat_cols=data.columns[(data.dtypes == 'object').values].tolist()
    for col in cat_cols:
        globals()['LE_{}'.format(col)] = LabelEncoder()
        data[col] = globals()['LE_{}'.format(col)].fit_transform(data[col])
    X=data.drop(columns='account')
       
    return X

#Devuelve las columnas que se usaron para entrenar el modelo 
def get_columns(dictionary):
    columns=[]
    for i in dictionary.keys():
        if dictionary[i]['use']==1:
            columns.append(i)
        else:
            pass
    return columns

In [6]:
with open('Models/{}/{}.json'.format(modelo,modelo),'r') as fp:
    json_=json.loads(fp.read())
    #modificar
with open('Models/{}/{}_prov.pkl'.format(modelo,modelo), 'rb') as f:
    model = pickle.load(f)
print(f"{bcolors.OKCYAN}Pickle y json listos de {modelo}")

Pickle y json listos de Starzplay


In [7]:
if TARGET[modelo]['input_gen']==0:
    print(f'{bcolors.ENDC}Leyendo datos ...')
    df=pd.read_csv('s3://uploadtp/BI/archivos_subir/CARTERAS_TXT_2/product_holding_model_feed.csv000',dtype={'account':'str'})
    #df=df.merge(asistente, left_on='account',right_on='cuenta',how='left')
    #df=df.merge(infantil, left_on='account', right_on='cuenta', how='left')
    #df['min_infantil'].fillna(0,inplace=True)
    #df['asistente_virtual'].fillna(0,inplace=True)

    megas=pd.read_csv('s3://uploadtp/BI/modelos/rf_megas/input/ph_megas.csv',names=['account','plan_megas','AAA_megas'],
                  dtype={'account':'str'})
    df=df.merge(megas,left_on='account',right_on='account',how='left')
    acc=df['account']
    
    print('Limpiando datos ...')
    df=limpieza(df,moda,fill_0,prom,dropr)
    print('Encoding ...')
    X_=encoding(df,tar)
    columnas=get_columns(json_)
    X=X_[columnas]
    
    print('Estandarizando ...')
    sc = StandardScaler()
    X_ = sc.fit_transform(X) 
    print('Data lista \n')
    print(f"{bcolors.OKBLUE}Iniciando predicciones para {modelo}...")
    predicted_proba=model.predict_proba(X_)
    print('...')
    if TARGET[modelo]['class']=='multi':
        predict=pd.DataFrame(predicted_proba, columns=['{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo),
                                                      '{}_proba_2'.format(modelo),'{}_proba_3'.format(modelo)])
        predict['account']=acc
        predict=predict[['account','{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo),
                         '{}_proba_2'.format(modelo),'{}_proba_3'.format(modelo)]]
        predict.to_csv('Outputs/predicciones_{}_{}.csv'.format(modelo,mes))
        print(f"{bcolors.OKGREEN}Predicciones listas para {modelo} :) ")
    
    else:
        predict=pd.DataFrame(predicted_proba, columns=['{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo)])
        predict['account']=acc
        predict=predict[['account','{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo)]]
        predict.to_csv('Outputs/predicciones_{}_{}.csv'.format(modelo,mes))
        print(f"{bcolors.OKGREEN}Predicciones listas para {modelo} :) ")
    

elif TARGET[modelo]['input_gen']==1:
    print(f'{bcolors.ENDC}Leyendo datos...')
    ph=pd.read_csv("s3://uploadtp/BI/Modelos_logs/INPUT/ago_21/ph_model_agos_clean.csv",dtype={'account':'str'})
    columnas=get_columns(json_)
    X=ph[columnas]
    print(f"{bcolors.OKBLUE}Iniciando predicciones para {modelo}...")
    predicted_proba=model.predict_proba(X_)
    print(f'{bcolors.ENDC}...')
    predict=pd.DataFrame(predicted_proba, columns=['{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo)])
    predict['account']=acc
    predict=predict[['account','{}_proba_0'.format(modelo),'{}_proba_1'.format(modelo)]]
    predict['{}'.format(modelo)]=np.where(predict['{}_proba_1'.format(modelo)]>=0.5,1,0)
    predict.to_csv('Outputs/predicciones_{}.csv'.format(modelo))
    print(f"{bcolors.OKGREEN}Predicciones listas para {modelo} :) ")
    


Leyendo datos ...


/home/ubuntu/BI/bi-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (26,27,28,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,50,52,66,68,70,72) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Limpiando datos ...
Encoding ...
Estandarizando ...
Data lista 

Iniciando predicciones para Starzplay...
...
Predicciones listas para Starzplay :) 


In [8]:
predict


,account,Starzplay_proba_0,Starzplay_proba_1
0,1.3334762,0.288835,0.711165
1,1.3377716,0.602960,0.397040
2,1.3391885,0.260364,0.739636
3,1.3423290,0.338010,0.661990
4,1.3427032,0.293957,0.706043
...,...,...,...
2932830,0106874129,0.773125,0.226875
2932831,0106874397,0.761182,0.238818
2932832,0106874523,0.870084,0.129916
2932833,0106874564,0.753256,0.246744
